## Creating connection to Redshift

In [78]:
import os
import sys
import psycopg2
import pandas as pd
from time import time

working_dir = os.getcwd()

connection_info_path = os.path.dirname(working_dir) + '/plugins/helpers/'

sys.path.append(connection_info_path)

import connection_info

print("Creating connection to Redshift")
redshift_credentials = next(item for item in connection_info.ConnectionInfo['connections']\
                          if item["conn_id"] == "redshift")
conn = psycopg2.connect("host={} dbname={} user={} password={} port={}"
                            .format(redshift_credentials['host'], redshift_credentials['schema'],
                                    redshift_credentials['login'], redshift_credentials['password'],
                                    redshift_credentials['port']))
cursor = conn.cursor()
print("Connection created")

Creating connection to Redshift
Connection created


## Getting table names and count of records

In [79]:
query  = """
select distinct t.table_name
from information_schema.tables t
where t.table_schema = 'public'
      and t.table_type = 'BASE TABLE'
order by t.table_name;
"""
cursor.execute(query)
table_names = cursor.fetchall()
table_name_with_count = []
for table in table_names:
    query = f"select count(*) from public.{table[0]}"
    cursor.execute(query)
    row = cursor.fetchall()
    table_name_with_count.append([table[0], row[0][0]])
    
df = pd.DataFrame(table_name_with_count, columns = ['Table_name','Count_of_records']) 
df

,Table_name,Count_of_records
0,covid_data_non_us,148757
1,covid_data_us,791811
2,google_mobility_data_non_us,2008372
3,google_mobility_data_us,723326
4,location_lookup,12540


In [80]:
## Getting count of distinct locations for which we have covid data and google mobility data


table_name_with_count = []
for table_name in table_names:
    if table_name[0] != 'location_lookup':
        query  = f"""
                    select count(distinct location_identifier||file_date)
                    from {table_name[0]}
                 """
    else:
        query  = f"""
                    select count(distinct location_identifier)
                    from {table_name[0]}
                 """        
    cursor.execute(query)
    row = cursor.fetchall()
    table_name_with_count.append([table_name[0], row[0][0]])
    
df = pd.DataFrame(table_name_with_count, columns = ['Table_name','Count_Of_Distinct_primary_key']) 
df

,Table_name,Count_Of_Distinct_primary_key
0,covid_data_non_us,148757
1,covid_data_us,791811
2,google_mobility_data_non_us,2008372
3,google_mobility_data_us,723326
4,location_lookup,12540


## Getting the most recent date of data availability

In [81]:
print(f"Table_name: max(file_date)")
for table in table_names:
    if table[0] != 'location_lookup':
        query = f"""select max(file_date) from public.{table[0]}"""
        cursor.execute(query)
        row = cursor.fetchall()
        print(f"{table[0]} : {row[0][0]}")

Table_name: max(file_date)
covid_data_non_us : 2020-11-29
covid_data_us : 2020-11-29
google_mobility_data_non_us : 2020-11-27
google_mobility_data_us : 2020-11-27


## Getting countries with max number of covid confirmed cases 

In [82]:
query = """select 
        ll.country_region as Country,
        sum(new_confirmed_cases) as total_cases
from covid_data_non_us cd
left join location_lookup ll
on ll.location_identifier = cd.location_identifier
WHERE cd.location_granularity = 'country'
group by ll.country_region
order by sum(new_confirmed_cases) desc
limit 5"""
cursor.execute(query)

rows = cursor.fetchall()

df = pd.DataFrame(rows,columns = ['Country','total_cases'])
df

,Country,total_cases
0,France,2217555
1,Argentina,1418807
2,Poland,985075
3,Iran,948749
4,South Africa,787702


## Getting US state with max number of covid confirmed cases

In [83]:
query = """
select 
        ll.county_or_city as county,
        ll.province_or_state as state,
        sum(new_confirmed_cases) as ttl_confirmed_cases
from covid_data_us cd
left join location_lookup ll
on ll.location_identifier = cd.location_identifier
WHERE cd.location_granularity = 'county'
group by ll.county_or_city, ll.province_or_state, ll.country_region
order by sum(new_confirmed_cases) desc
limit 10;"""

cursor.execute(query)

rows = cursor.fetchall()

df = pd.DataFrame(rows,columns = ['county','state','ttl_confirmed_cases'])
df

,county,state,ttl_confirmed_cases
0,Los Angeles,California,395843
1,Cook,Illinois,304220
2,Miami-Dade,Florida,227656
3,Maricopa,Arizona,205332
4,Harris,Texas,190631
5,Dallas,Texas,137096
6,Clark,Nevada,115190
7,Broward,Florida,106698
8,Tarrant,Texas,97294
9,San Bernardino,California,93019
